## Python Web Scraper 

In [216]:
# importing libraries
import requests

from requests import get
import lxml
from lxml.html.soupparser import fromstring
import prettify
import numbers
import html_text

import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import sys
import numpy as np
import pandas as pd
import regex as re

from urllib.request import urlopen
from lxml import html

In [210]:
# both functions call find_all method on soup object to filter through the div class
def select_properties(soup):
    res = soup.find_all('div', id='grid-search-results')
    properties = res[0].find_all('ul')[0].find_all('li')
    return properties

def select_res(soup):
    res = soup.find_all('div', id='grid-search-results')
    return res

In [211]:
def get_p(res):
# filters through the html script and extracts price values
    prices = list()
    
    for val in res:
        pc = val.find_all(class_='list-card-price')
        #print(pc)
        #print(val)
        pc = re.sub('<div class="list-card-price">', '', str(pc))
        pc = re.sub('</div>', '', pc)
        pc = pc.replace('[','').replace(']','')
        pc = pc.replace(',','')
        pc = pc.split()
        #print(type(pc), "Hi")
        #print(pc)
        if len(pc) != 0:
            prices.append(pc)
        
    flat_pc = [item for sublist in prices for item in sublist] #flatten the nested list of prices
        
    return flat_pc
    

def get_clean_price(prices):
    #append price value to new list
    
    int_price = list() #change back to nprice later
    
    for i in prices:
        i = i.replace("''", '')
        ni = re.sub('\$', '', i)
        
        #print(ni)
        #print(ni)
        int_price.append(ni)
    
    #int_price = [int(i) for i in nprices]
    return int_price

In [212]:
def get_rent_p(res):
# filters through the html script and extracts price values
    prices = list()
    
    for val in res:
        pc = val.find_all(class_='list-card-price')
        #print(pc)
        #print(val)
        pc = re.sub('<div class="list-card-price">', '', str(pc))
        pc = re.sub('</div>', '', pc)
        pc = pc.replace('[','').replace(']','')
        pc = pc.replace(',','')
        pc = pc.split()
        #print(type(pc), "Hi")
        #print(pc)
        if len(pc) != 0: #sets condition to check for whether list has actual values inside and is not empty
            prices.append(pc)
        
    flat_pc = [item for sublist in prices for item in sublist] #flatten the nested list of prices
        
    return flat_pc
    
def get_clean_rent(prices):
    #append price value to new list
    
    int_price = list() #change back to nprice later
    
    for i in prices:
        i = i.replace("''", '')
        ni = re.sub('\$', '', i)
        
        #print(ni)
        #print(ni)
        int_price.append(ni)
    
    #int_price = [int(i) for i in nprices]
    return int_price

In [213]:
def get_address(properties):

    wb = list()

    # finds the address header and cleans up all the html script
    for prop in properties:
        house = prop.find_all(class_='list-card-info')
        if len(house) != 0:
            address = house[0].find('address')
            if address is not None: # handling None type objects
                ad = re.sub('<address class="list-card-addr">', ' ', str(address))
                clean_ad = re.sub('</address>', ' ', ad)
                print(clean_ad)
            #sites = a.attrs['href']
                wb.append(clean_ad)
            else:
                continue
                
    return wb
                

In [214]:
# extracts the city and zip code from the address and creates two new lists for the city and zip code separately

def city(wb):

    city = [sub.split(',')[1] for sub in wb]
    fcity = [sub.strip(" ") for sub in city]
    return fcity

def zip_code(wb):
    zc = [sub.split(',')[2] for sub in wb]
    zcode = [sub.replace('MA', '') for sub in zc]
    fzc = [sub.strip(" ") for sub in zcode]
    #zip_code = [int(i) for i in fzc]
    return fzc


In [215]:
def get_url(properties):

    info = list()

    # finds the url header and cleans up all the html script
    for prop in properties:
        house = prop.find_all(class_='list-card-info')
        #print(house)
        if len(house) != 0: 
            ac = house[0].find('a')
            #print(ac)
            if ac is not None:
                #print(a)
                sites = ac.attrs['href']
                info.append(sites)
            else:
                continue
            
    return info

In [179]:
def find_beds(soup):
    beds = soup.find_all("ul", class_="list-card-details") # specific class where bed values are located 
    return beds

def get_bed_bath_sqft(beds):

    lst = list()

    # takes the bed, bath, and square feet values and creates a nested list with each of the three values split  
    for item in beds:
        prop = item.find_all('li')
        p1 = re.sub('<li class="">', '', str(prop))
        p1 = re.sub('<abbr class="list-card-label">', '', p1)
        p1 = re.sub('<!-- -->', '', p1)
        p1 = re.sub('</abbr></li>', '', p1)
        p1 = re.sub(', <li class="list-card-statusText">-', '', p1)
        p1 = re.sub('</li>', '', p1)
        p1 = re.sub('Condo for sale', '', p1)
        p1 = re.sub('Multi-family home for sale', '', p1)
        p1 = re.sub('House for sale', '', p1)
        p1 = p1.replace('[','').replace(']','')
        p1 = re.sub('bds', '', p1)
        p1 = re.sub('bd', '', p1)
        p1 = re.sub('ba', '', p1)
        p1 = re.sub('sqft', '', p1)
        p1 = re.sub('Sold', '', p1)
        p1 = p1.replace(' ', '')
        p1 = p1.split(',', 2)
        lst.append(p1)
    
    return lst

In [180]:
# list comprehension to create separate list for each bed, bath, and sqft value in the nested list above
def get_bed(lst):
    bed = [i[0] for i in lst] 
    return bed

def get_bath(lst):
    bath = [i[1] for i in lst]
    return bath

def get_sqft(lst):
    sqft = [i[2] for i in lst]
    return sqft

In [181]:
def create_df(int_price, wb, fcity, fzc, bed, bath, sqft, info):
    # create pandas series for each list of attributes and combines them into a dataframe
    int_price = pd.Series(int_price) 
    wb = pd.Series(wb)
    fcity = pd.Series(fcity)
    fzc = pd.Series(fzc)
    bed = pd.Series(bed)
    bath = pd.Series(bath)
    sqft = pd.Series(sqft)
    info = pd.Series(info)

    df=pd.concat([int_price, wb, fcity, fzc, bed, bath, sqft, info],axis=1)
    df.columns = ['Price', 'Address', 'City', 'Zip Code', 'Beds', 'Bath', 'Square Feet', 'Links']
    df.head(10)
    
    df = df.replace('--', np.nan) # replace any null values 
    
    return df

def filter_df(df):
# replaces any letter values with string, converts it to an integer
#     repl_dict = {'[kK]': '*1e3', '[mM]': '*1e6', '[bB]': '*1e9', }
#     df['Price'].replace(repl_dict, regex=True).map(pd.eval)
    df.Price = (df.Price.replace(r'[KMB]+$', '', regex=True).astype(float) * 
              df.Price.str.extract(r'[\d\.]+([KMB]+)', expand=False).fillna(1).replace(['K','M', 'B'], 
                                                                                     [10**3, 10**6, 10**9]))
    df['Price'] = df['Price'].astype(int)
    return df

In [144]:
# test urls 
url_1 = "https://www.zillow.com/homes/Boston,-MA_rb/"
url_2 = "https://www.zillow.com/boston-ma/2_p/"
url_3 = "https://www.zillow.com/boston-ma/3_p/" 
url_4 = "https://www.zillow.com/boston-ma/4_p/"

In [196]:
curr = 1

In [217]:
# Url for buying houses
url = f"https://www.zillow.com/boston-ma/{curr}_p/"
print(curr)

2


In [183]:
# Url for house rentals
burl = f"https://www.zillow.com/boston-ma/rentals/{curr}_p/"
print(curr)

1


In [172]:
#Url for houses sold
curl = f"https://www.zillow.com/boston-ma/sold/{curr}_p/"
print(curr)

24


In [218]:
chromedriver = "/opt/homebrew/bin/chromedriver" # path to the chromedriver executable
#chromedriver = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
chromedriver = os.path.expanduser(chromedriver)
print('chromedriver path: {}'.format(chromedriver))
sys.path.append(chromedriver)
driver = webdriver.Chrome(chromedriver)
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

# DONT FORGET TO UNCOMMENT THIS SHIT AFTER U RUN 12!!!

curr += 1

chromedriver path: /opt/homebrew/bin/chromedriver


/var/folders/t4/x2nfgfvj7ts940mwzqzt0htr0000gn/T/ipykernel_82130/2175428446.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


We were unable to run any type of loop or function call that would take the current url and add 1 to the page section due to Zillow's antiscraping techniques. So, we built the lines of code below that essentially start off by setting the current page value to 1 and then running the url once through the chromedriver before adding one to the curr, or current value to move to the next page.

In [114]:
print(curr)

15


In [ ]:
def get_rent_p(res):
# filters through the html script and extracts price values
    prices = list()
    
    for val in res:
        pc = val.find_all(class_='list-card-price')
        #print(pc)
        #print(val)
        pc = re.sub('<div class="list-card-price">', '', str(pc))
        pc = re.sub('</div>', '', pc)
        pc = pc.replace('[','').replace(']','')
        pc = pc.replace(',','')
        pc = pc.split()
        #print(type(pc), "Hi")
        #print(pc)
        if len(pc) != 0:
            prices.append(pc)
        
    flat_pc = [item for sublist in prices for item in sublist] #flatten the nested list of prices
        
    return flat_pc
    
def get_clean_rent(prices):
    #append price value to new list
    
    int_price = list() #change back to nprice later
    
    for i in prices:
        i = i.replace("''", '')
        ni = re.sub('\$', '', i)
        
        #print(ni)
        #print(ni)
        int_price.append(ni)
    
    #int_price = [int(i) for i in nprices]
    return int_price

In [188]:
# this one 
prop1 = select_properties(soup)
print(prop1)

[<li><article class="list-card list-card-additional-attribution list-card_not-saved list-card_building" id="zpid_42.36566--71.06165" role="presentation"><div class="list-card-info"><a class="list-card-link list-card-link-top-margin" href="/b/hub50house-boston-ma-9qFF3Z/" tabindex="0"><address class="list-card-addr">Hub50House | 50 Causeway St, Boston, MA</address></a><div class="list-card-footer"></div><div class="list-card-heading"><div class="list-card-price">$3,010+<abbr class="list-card-label"> <!-- -->Studio</abbr></div><ul class="list-card-details"><li class="">$3,553+<abbr class="list-card-label"> <!-- -->1 bd</abbr></li><li class="">$5,675+<abbr class="list-card-label"> <!-- -->2 bds</abbr></li><li class="">$9,483+<abbr class="list-card-label"> <!-- -->3 bds</abbr></li></ul></div></div><div class="list-card-top"><div class="list-card-variable-text list-card-img-overlay">Updated yesterday</div><a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" 

In [187]:
res1 = select_res(soup)
print(res1)

[<div class="result-list-container" id="grid-search-results" role="main"><div class="search-page-list-header"><h1 class="search-title">Boston MA Rental Listings</h1><div class="srp__sc-1h8ajh8-1 dxXOox search-subtitle"><span class="result-count">5,884 results</span><div class="srp__sc-5mqxuh-0 jifEpS sort-options visible"><strong id="sort_label">Sort by:</strong><button aria-expanded="false" aria-haspopup="menu" aria-label="Sort options, Default selected" class="StyledButton-c11n-8-62-4__sc-wpcbcc-0 jovTww DropdownButton-c11n-8-62-4__sc-i5s403-0 srp__sc-5mqxuh-1 hBKzEE DropdownPopper-c11n-8-62-4__sc-1vnow1h-0 fualOl MenuPopper-c11n-8-62-4__sc-1uu201g-0 fGOhWU a11y-pagination-focus" id="sort-popover">Default<span class="StyledButtonIcon-c11n-8-62-4__sc-wpcbcc-1 dRUWRb"><svg aria-hidden="true" class="Icon-c11n-8-62-4__sc-13llmml-0 jXXRjv" focusable="false" role="img" viewbox="0 0 32 32"><title>Chevron Down</title><path d="M29.7 8.8a1 1 0 00-1.4 0L16 21 3.7 8.8a1 1 0 00-1.4 0 1 1 0 000 1.

In [189]:
raw_price = get_p(res1)
print(raw_price)
int_price = get_clean_price(raw_price)
print(int_price)

['$3010+<abbr', 'class="list-card-label">', '<!--', '-->Studio</abbr>', '$3278+<abbr', 'class="list-card-label">', '<!--', '-->Studio</abbr>', '$3269+<abbr', 'class="list-card-label">', '<!--', '-->Studio</abbr>', '$3253+<abbr', 'class="list-card-label">', '<!--', '-->Studio</abbr>', '$3754/mo', '$2389+<abbr', 'class="list-card-label">', '<!--', '-->Studio</abbr>', '$2560+<abbr', 'class="list-card-label">', '<!--', '-->Studio</abbr>', '$3250+<abbr', 'class="list-card-label">', '<!--', '-->1', 'bd</abbr>', '$2767+<abbr', 'class="list-card-label">', '<!--', '-->Studio</abbr>']
['3010+<abbr', 'class="list-card-label">', '<!--', '-->Studio</abbr>', '3278+<abbr', 'class="list-card-label">', '<!--', '-->Studio</abbr>', '3269+<abbr', 'class="list-card-label">', '<!--', '-->Studio</abbr>', '3253+<abbr', 'class="list-card-label">', '<!--', '-->Studio</abbr>', '3754/mo', '2389+<abbr', 'class="list-card-label">', '<!--', '-->Studio</abbr>', '2560+<abbr', 'class="list-card-label">', '<!--', '-->St

In [190]:
info = get_url(prop1)
print(info)

['/b/hub50house-boston-ma-9qFF3Z/', '/b/The-Sudbury-Boston-MA/42.362183,-71.059654_ll/', '/b/one-greenway-boston-ma-65WmdH/', '/b/the-harlo-boston-ma-5fNMSs/', 'https://www.zillow.com/homedetails/530-Atlantic-Ave-APT-202-Boston-MA-02210/2090018407_zpid/', '/b/eco-allston-ma-5XksSc/', '/b/the-victor-by-windsor-boston-ma-5XmCFX/', '/b/fenway-diamond-at-9-miner-street-boston-ma-5YwqXf/', '/b/radian-boston-ma-5XjQjN/']


In [202]:
# this one 
prop1 = select_properties(soup)
res1 = select_res(soup)
#print(res1)

raw_price = get_p(res1)
print(raw_price)
int_price = get_clean_price(raw_price)
print(int_price)

wb = get_address(prop1)
print(wb)

fcity = city(wb)
print(fcity)
fzc = zip_code(wb)
print(fzc)

info = get_url(prop1)
print(info)

finder = find_beds(soup)
bbs = get_bed_bath_sqft(finder)

bed = get_bed(bbs)
print(bed)
bath = get_bath(bbs)
print(bath)
sqft= get_sqft(bbs)
print(sqft)

['$599900', '$579900', '$749000', '$1649000', '$639000', '$450000', '$130000', '$265000', '$2699000']
['599900', '579900', '749000', '1649000', '639000', '450000', '130000', '265000', '2699000']
 84-86 Sunnyside St #2, Hyde Park, MA 02136 
 84-86 Sunnyside St #1, Hyde Park, MA 02136 
 19 Highfield Rd, Roslindale, MA 02131 
 8 Church St, Boston, MA 02116 
 110 Beacon St, Hyde Park, MA 02136 
 141 Chelsea St, East Boston, MA 02128 
 790 Hyde Park Ave APT 16, Boston, MA 02136 
 300 Summer St APT 23, Boston, MA 02210 
 51 Commercial Wharf APT 7, Boston, MA 02110 
[' 84-86 Sunnyside St #2, Hyde Park, MA 02136 ', ' 84-86 Sunnyside St #1, Hyde Park, MA 02136 ', ' 19 Highfield Rd, Roslindale, MA 02131 ', ' 8 Church St, Boston, MA 02116 ', ' 110 Beacon St, Hyde Park, MA 02136 ', ' 141 Chelsea St, East Boston, MA 02128 ', ' 790 Hyde Park Ave APT 16, Boston, MA 02136 ', ' 300 Summer St APT 23, Boston, MA 02210 ', ' 51 Commercial Wharf APT 7, Boston, MA 02110 ']
['Hyde Park', 'Hyde Park', 'Roslind

In [203]:
# this one 
df1 = create_df(int_price, wb, fcity, fzc, bed, bath, sqft, info)
df1

,Price,Address,City,Zip Code,Beds,Bath,Square Feet,Links
0,599900,"84-86 Sunnyside St #2, Hyde Park, MA 02136",Hyde Park,02136,3,2,"1,449",https://www.zillow.com/homedetails/84-86-Sunny...
1,579900,"84-86 Sunnyside St #1, Hyde Park, MA 02136",Hyde Park,02136,2,2,"1,134",https://www.zillow.com/homedetails/84-86-Sunny...
2,749000,"19 Highfield Rd, Roslindale, MA 02131",Roslindale,02131,3,2,"1,643",https://www.zillow.com/homedetails/19-Highfiel...
3,1649000,"8 Church St, Boston, MA 02116",Boston,02116,4,4,"3,000",https://www.zillow.com/homedetails/8-Church-St...
4,639000,"110 Beacon St, Hyde Park, MA 02136",Hyde Park,02136,4,3,"1,703",https://www.zillow.com/homedetails/110-Beacon-...
5,450000,"141 Chelsea St, East Boston, MA 02128",East Boston,02128,4,3,"3,023",https://www.zillow.com/homedetails/141-Chelsea...
6,130000,"790 Hyde Park Ave APT 16, Boston, MA 02136",Boston,02136,1,1,655,https://www.zillow.com/homedetails/790-Hyde-Pa...
7,265000,"300 Summer St APT 23, Boston, MA 02210",Boston,02210,1,1,"1,220",https://www.zillow.com/homedetails/300-Summer-...
8,2699000,"51 Commercial Wharf APT 7, Boston, MA 02110",Boston,02110,3,2,"1,879",https://www.zillow.com/homedetails/51-Commerci...


In [170]:
final = filter_df(df1)
final.head(10)
#df2 = create_df(int_price, wb, fcity, fzc, bed, bath, sqft, info)

,Price,Address,City,Zip Code,Beds,Bath,Square Feet,Links
0,767000,"691 Massachusetts Ave UNIT 208, Boston, MA 02...",Boston,02118,1,1,909,https://www.zillow.com/homedetails/691-Massach...
1,560000,"104 King St #3, Dorchester, MA 02122",Dorchester,02122,2,1,"1,007",https://www.zillow.com/homedetails/104-King-St...
2,944000,"241 Perkins St UNIT J101, Jamaica Plain, MA 0...",Jamaica Plain,02130,3,3,"2,190",https://www.zillow.com/homedetails/241-Perkins...
3,1050000,"25 Bay State Rd STE 2, Boston, MA 02215",Boston,02215,3,3,"1,124",https://www.zillow.com/homedetails/25-Bay-Stat...
4,580000,"59B Old Morton St, Boston, MA 02126",Boston,02126,3,3,"1,300",https://www.zillow.com/homedetails/59B-Old-Mor...
5,725000,"47 Goodale Rd #1, Mattapan, MA 02126",Mattapan,02126,3,2,"1,322",https://www.zillow.com/homedetails/47-Goodale-...
6,2560000,"50 Liberty Dr PENTHOUSE 1J, Boston, MA 02210",Boston,02210,1,2,"1,329",https://www.zillow.com/homedetails/50-Liberty-...
7,1070000,"18 Brenton St, Dorchester, MA 02121",Dorchester,02121,7,4,"4,685",https://www.zillow.com/homedetails/18-Brenton-...
8,410000,"4975 Washington St #118, West Roxbury, MA 02132",West Roxbury,02132,2,2,"1,055",https://www.zillow.com/homedetails/4975-Washin...


## Purchasing Houses

In [207]:
# Convert df to csv and export to store and update data
# Only run this line of code one time to import the headers

final.to_csv('purchase_zillow_house3000.csv', mode='a', index=False)
print("Housing data appended successfully.")

Housing data appended successfully.


In [208]:
# Remove headers when adding every other time to avoid duplicate header row
final.to_csv('purchase_zillow_house3000.csv', mode='a', header=False, index=False)
print("Housing data appended successfully.")

Housing data appended successfully.


## Rental Data

In [205]:
# Separate df and csv file created for rental data
# Only run this line of code one time to import the headers

df1.to_csv('rental_zillow_house.csv', mode='a', index=False)
print("Rental data appended successfully.")

Rental data appended successfully.


In [206]:
# Remove headers when adding every other time to avoid duplicate header row
df1.to_csv('rental_zillow_house.csv', mode='a', header=False, index=False)
print("Rental data appended successfully.")

Rental data appended successfully.
